# 1Dimensional Convolution Neural Network Long Short Term Memory (1D-CNN LSTM)

**what is 1D-CNN LSTM?**

- Extracts and identifies key features from sequential data to enhance pattern recognition for improved model performance.

- 1D-CNN Model is Used in the many Applications, Including Time Series Analysis, Signal Processing , Text Classifications etc..

**Key Features:**
- Spatial Feature Helps to identify key features and trends from Different station data that are important for making accurate predictions.

- Captures the Temporal Features from the data and Analyzes the changes and patterns over time within the data.

- Feature Fusion will Combine all features into one feature vector.

- Let’s see the Structure of the 1D-CNN Model.


The Below are the Required Modules for this Model

In [1]:
# Required Models for the project

import pandas as pd # type: ignore
import numpy as  np # type: ignore 
import os # type: ignore 
import math # type: ignore
import matplotlib.pyplot as plt # type: ignore

from sklearn.preprocessing import MinMaxScaler # type: ignore 
from tensorflow.keras.models import Model ,save_model ,load_model # type: ignore
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Flatten, concatenate # type: ignore
from tensorflow.keras.losses import MeanSquaredError # type: ignore
from tensorflow.keras.optimizers import Adam # type: ignore
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error # type: ignore

Now, we have to load our dataset required for the Model

In [ ]:
file_path_1 = 'PATH_TO_YOUR_DATASET.csv' # Path to your dataset
station1 = pd.read_csv(file_path_1) # Load the dataset
file_path_2 = 'PATH_TO_YOUR_DATASET.csv' # Path to your dataset
station2 = pd.read_csv(file_path_2) # Load the dataset
file_path_3 = 'PATH_TO_YOUR_DATASET.csv' # Path to your dataset
station3 = pd.read_csv(file_path_3) # Load the dataset
'''
So on the number stations depends upon you we tried 
on the 4 stations and the results were good
feel free to experiment the model with more stations'''


We have to merge all the stations data to the 1 dataset.

So, take the required columns from the dataset and combine the datasets to 1 dataset.

In [ ]:
combined_station = station1.merge(station2, on='Time Steps', suffixes=('_1', '_2'))
combined_station = combined_station.merge(station3, on='Time Steps', suffixes=('', '_3'))
Flows = ['Time Steps', 'Flow_1', 'Flow_2', 'Flow_3']
combined_station = combined_station[Flows]
combined_station.head()

To make sure There is no zero values in the dataset,we have to check is there any null values in the dataset.

In [ ]:
null_values = combined_station.isnull().sum() # Check for null values
print("Null values in each column:\n", null_values ) # Print the null values

Now, We have to get the required data values in between 0 and 1 to make the caculations simpler.

So, we are using the Normalizatioin Method.

We have to use 2 Scalers.

- 1 scaler to fit the 4 stations combined dataset to get the spatial features from the 4 stations data.

- 2 scaler to fit the point of intrest data set only Since we are going to predict the values in that stations.

In [ ]:
target_scaler = MinMaxScaler()
column1= ['Column Name']

my_station_df = pd.DataFrame(columns=column1)
my_station_df[column1] = combined_station[column1]

my_station_df[column1] = target_scaler.fit_transform(my_station_df[column1])
my_station_df.head()

In [ ]:
column = ['Flow_1', 'Flow_2', 'Flow_3']
scaler = MinMaxScaler()

combined_station[column] = scaler.fit_transform(combined_station[column])
combined_station.head()

# Sequence Creation

We have to divide the data into the sequences according to the required number of TimeIntervals.

In [ ]:
# Function to create sequences

def create_sequences(column_data, time_steps):
    X, Y = [], []
    for i in range(len(column_data) - time_steps):
        X.append(column_data[i:i + time_steps])
        Y.append(column_data[i + time_steps])
    return np.array(X), np.array(Y)

In [ ]:
column_data = combined_station[['Flow_1', 'Flow_2', 'Flow_3']].values
time_steps = 'No_of_time_steps' # Number of time steps   
input_dimension = 1 # Input dimension

CNN_X, _ = create_sequences(column_data, time_steps)

station1_data = column_data[:, 0].reshape(-1, 1)
LSTM_X, y = create_sequences(station1_data, time_steps)

# DISPLAYING THE VALUES OF THE VARIABLES

print(f"Values of cnn_X: {CNN_X}")
print(f"Values of lstm_X: {LSTM_X}")
print(f"Values of y: {y}")

print(f"Shape of cnn_X: {CNN_X.shape}")
print(f"Shape of lstm_X: {LSTM_X.shape}")
print(f"Shape of y: {y.shape}")

# Spliting the Data 

Now , we have to split the data for training and testing the data.

In [ ]:
# Splitting the data into training top 90% and testing bottom 10%

train_size = int(len(CNN_X) * 0.90)
test_size = len(CNN_X) - train_size

X_train_cnn, X_test_cnn = CNN_X[:train_size], CNN_X[train_size:]
X_train_lstm, X_test_lstm = LSTM_X[:train_size], LSTM_X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Displaying the shapes of the training and testing data

print(X_train_cnn.shape)
print(X_train_lstm.shape)
print(X_test_cnn.shape)
print(X_test_lstm.shape)
print(y_train.shape)
print(y_test.shape)

print("Training CNN X Values : ",X_train_cnn)
print("Training LSTM X Values : ",X_train_lstm)
print("Testing CNN X Values : ",X_test_cnn)
print("Testing LSTM X Values : ",X_test_lstm)
print("Training y Values : ",y_train)
print("Testing y Values : ",y_test)

# 1D-CNN LSTM Model Creation

We have to create the 1D-CNN LSTM Model as we already imported the required modeules respectively.

In [ ]:
cnn_input = Input(shape=(time_steps, 4)) # 4 is the number of features which is equal to the number of stations we used in the dataset
cnn_layer = Conv1D(filters="number of filters Required", kernel_size=2, activation='')(cnn_input) # Number of filters depends upon you
cnn_layer = Flatten()(cnn_layer) # Flatten the layer

lstm_input = Input(shape=(time_steps, 1)) # 1 is the number of features which is the point of interest we are going to predict the model 
lstm_layer = LSTM("number of neurons required", activation='', return_sequences=False)(lstm_input) # Number of neurons depends upon you

combined = concatenate([cnn_layer, lstm_layer]) # Concatenate the layers
output = Dense(1)(combined) # Dense layer with 1 neuron

model = Model(inputs=[cnn_input, lstm_input], outputs=output) # Create the model

model.compile(optimizer='', loss='') # Compile the model

model.summary() # Display the model summary

We created the model for the 1D-CNN LSTM, Now we have to train the model by using the Training Data.

In [ ]:
history = model.fit([X_train_cnn, X_train_lstm], y_train, epochs="number of times you want to train the model", validation_split="Menton the validaion split here") # Train the model

Upto to this is the model creation and the model training.

Let's Predict the values using the model we trained.

In [ ]:
predicted_values = model.predict([X_test_cnn , X_test_lstm] ) # Predict the values

predictions_rescaled = target_scaler.inverse_transform(predicted_values.reshape(-1, 1)).flatten() # Rescale the predictions
y_test_rescaled = target_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten() # Rescale the actual values

results = pd.DataFrame(data={'Predictions': predictions_rescaled, 'Actuals': y_test_rescaled}) # Create a dataframe of the results
print(results) # Display the results

# Saving the Trained Model

Now, we have to save the moedel so we can load the model without re-training the model.

In [ ]:
model.save("path_to_save_the_maodel") # Saving the model

In [ ]:
model1 = load_model("path_to_save_the_maodel") # Loading the model

# Loss Values of the Model 

Now , we have to check the loss values of the Model we trained so we can get the  accuracy of the model.

In [ ]:
loss = model.evaluate([X_test_cnn, X_test_lstm], y_test)
print(f'Test Loss: {loss}')

In [ ]:
rmse = math.sqrt(mean_squared_error(y_test_rescaled, predictions_rescaled))
mae = mean_absolute_error(y_test_rescaled, predictions_rescaled)
mape = mean_absolute_percentage_error(predictions_rescaled, y_test_rescaled)

# Displaying the RMSE, MAE and MAPE values

print('RMSE:', rmse)
print('MAE:', mae)
print('MAPE:', mape)

In [ ]:
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train', 'Validation'], loc='upper left')
plt.show()